## 16-04. RAG + Image Generator Agent(보고서 작성)
- 웹 검색(Web Search), PDF 문서 기반 검색(RAG), 이미지 생성(Image Generation) 등을 통해 보고서를 작성하는 에이전트

<br>



In [1]:
from dotenv import load_dotenv

load_dotenv()

True

<br>

#### 웹 검색도구: Tavily Search



In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [4]:
search = TavilySearchResults(k=6)

C:\Users\user\AppData\Local\Temp\ipykernel_37208\3100324165.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search = TavilySearchResults(k=6)


<br>

#### 문서 기반 문서 검색 도구: Retriever


In [5]:
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_classic.document_loaders import PyMuPDFLoader

In [6]:
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = loader.load_and_split(text_splitter)
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [8]:
from langchain_classic.tools.retriever import create_retriever_tool
from langchain_core.prompts import PromptTemplate

- `document_prompt` 는 문서의 내용을 표시하는 템플릿을 정의

In [10]:
document_prompt = PromptTemplate.from_template(
    "<document><content>{page_content}</content><page>{page}</page><filename>{source}</filename></document>"
)

In [11]:
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="use this tool to search for information in the PDF file",
    document_prompt=document_prompt,
)

<br>

#### DallE 이미지 생성 도구


In [12]:
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain.tools import tool

In [ ]:
dalle = DallEAPIWrapper(model="dall-e-3", size="1024x1024", quality="standard", n=1)

In [14]:
@tool
def dalle_tool(query):
    """use this tool to generate image from text"""
    return dalle.run(query)

<br>

#### 파일 관리 도구


In [15]:
from langchain_community.agent_toolkits import FileManagementToolkit

In [16]:
working_directory = "tmp"

# 파일 관리 도구 생성(파일 쓰기, 읽기, 디렉토리 목록 조회)
file_tools = FileManagementToolkit(
    root_dir=str(working_directory),
    selected_tools=["write_file", "read_file", "list_directory"],
).get_tools()

file_tools

[WriteFileTool(root_dir='tmp'),
 ReadFileTool(root_dir='tmp'),
 ListDirectoryTool(root_dir='tmp')]

<br>

#### 모든 도구를 종합

In [17]:
tools = file_tools + [
    retriever_tool,
    search,
    dalle_tool,
]

tools

[WriteFileTool(root_dir='tmp'),
 ReadFileTool(root_dir='tmp'),
 ListDirectoryTool(root_dir='tmp'),
 StructuredTool(name='pdf_search', description='use this tool to search for information in the PDF file', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x0000024B6B76D580>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x0000024B6B76DE40>),
 TavilySearchResults(api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))),
 StructuredTool(name='dalle_tool', description='use this tool to generate image from text', args_schema=<class 'langchain_core.utils.pydantic.dalle_tool'>, func=<function dalle_tool at 0x0000024B6E51B1A0>)]

<br>

### Agent 생성

<br>

#### 1. 문서기반 검색 + 파일 생성


In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_openai import ChatOpenAI

In [20]:
store = {}

In [21]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "You are a professional researcher. "
            "You can use the pdf_search tool to search for information in the PDF file. "
            "You can find further information by using search tool. "
            "You can use image generation tool to generate image from text. "
            "Finally, you can use file management tool to save your research result into files.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [22]:
llm = ChatOpenAI(model="gpt-4o-mini")
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    handle_parsing_errors=True,
)

In [23]:
def get_session_history(session_ids):
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]

In [24]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

<br>

- **에이전트 실행**

In [25]:
result = agent_with_chat_history.stream(
    {
        "input": "삼성전자가 개발한 `생성형 AI` 와 관련된 유용한 정보들을 PDF 문서에서 찾아서 bullet point로 정리해 주세요. "
        "한글로 작성해주세요."
        "다음으로는 `report.md` 파일을 새롭게 생성하여 정리한 내용을 저장해주세요. \n\n"
        "#작성방법: \n"
        "1. markdown header 2 크기로 적절한 제목을 작성하세요. \n"
        "2. 발췌한 PDF 문서의 페이지 번호, 파일명을 기입하세요(예시: page 10, filename.pdf). \n"
        "3. 정리된 bullet point를 작성하세요. \n"
        "4. 작성이 완료되면 파일을 `report.md` 에 저장하세요. \n"
        "5. 마지막으로 저장한 `report.md` 파일을 읽어서 출력해 주세요. \n"
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    print(step)

Agent 실행 결과:
{'actions': [ToolAgentAction(tool='pdf_search', tool_input={'query': '삼성전자 생성형 AI'}, log="\nInvoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model_provider': 'openai', 'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_414ba99a04', 'service_tier': 'default'}, id='lc_run--019c89bb-8d92-73f1-9210-41f108efe773', tool_calls=[{'name': 'pdf_search', 'args': {'query': '삼성전자 생성형 AI'}, 'id': 'call_XDFlrf4srxjB0SDuaWYa6FT5', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 479, 'output_tokens': 19, 'total_tokens': 498, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}, tool_call_chunks=[{'name': 'pdf_search', 'args': '{"query":"삼성전자 생성형 AI"}', 'id': 'call_XDFlrf4srxjB0SDuaWYa6FT5', 'index': 0, 'type': 'tool_call_chunk'}], chunk_position='last')], tool_call_i

KeyboardInterrupt: 

<br>

#### 2. 웹 검색 + 파일 수정
- 생성된 보고서 파일 (`report.md`)의 내용을 업데이트


In [ ]:
result = agent_with_chat_history.stream(
    {
        "input": "이번에는 삼성전자가 개발한 `생성형 AI` 와 관련된 정보들을 웹 검색하고, 검색한 결과를 정리해 주세요. "
        "한글로 작성해주세요."
        "다음으로는 `report.md` 파일을 열어서 기존의 내용을 읽고, 웹 검색하여 찾은 정보를 이전에 작성한 형식에 맞춰 뒷 부분에 추가해 주세요. \n\n"
        "#작성방법: \n"
        "1. markdown header 2 크기로 적절한 제목을 작성하세요. \n"
        "2. 정보의 출처(url)를 기입하세요(예시: 출처: 네이버 지식백과). \n"
        "3. 정리된 웹검색 내용을 작성하세요. \n"
        "4. 작성이 완료되면 파일을 `report.md` 에 저장하세요. \n"
        "5. 마지막으로 저장한 `report.md` 파일을 읽어서 출력해 주세요. \n"
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    print(step)

<br>

#### 3. 보고서 작성

In [ ]:
result = agent_with_chat_history.stream(
    {
        "input": "`report.md` 파일을 열어서 안의 내용을 출력하세요. "
        "출력된 내용을 바탕으로, 전문적인 수준의 보고서를 작성하세요. "
        "보고서는 총 3개의 섹션으로 구성되어야 합니다:\n"
        "1. 개요: 보고서 abstract 를 300자 내외로 작성하세요.\n"
        "2. 핵심내용: 보고서의 핵심 내용을 작성하세요. 정리된 표를 markdown 형식으로 작성하여 추가하세요. "
        "3. 최종결론: 보고서의 최종 결론을 작성하세요. 출처(파일명, url 등)을 표시하세요."
        "마지막으로 작성된 결과물을 `report-2.md` 파일에 저장하세요."
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    print(step)

<br>

#### 보고서 내용을 기반으로 이미지 생성을 요청

In [ ]:
result = agent_with_chat_history.stream(
    {
        "input": "`report-2.md` 파일을 열어서 안의 내용을 출력하세요. "
        "출력된 내용에 어울리는 이미지를 생성하세요. "
        "생성한 이미지의 url 을 markdown 형식으로 보고서의 가장 상단에 추가하세요. "
        "마지막으로 작성된 결과물을 `report-3.md` 파일에 저장하세요."
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    print(step)

<br>

<hr>